In [111]:
import re
def clean(text):
  text = " ".join(text.split())
  text = text.replace("[[", "")
  text = text.replace("]]", "")
  text = text.replace("\n", "")
  text = text.replace("*", "")
  return text

def clean_dict(dictionary):
  d = {}
  for k, v in dictionary.items():
    if v != {}:
      d[k] = {}
    if  isinstance(v, list):
        d[k] = v
    else:
      for k2, v2 in v.items():
        if v2:
            d[k][k2] = v2
  return d

In [112]:
def read_lines(filename):
    with open(f"convert_files/{filename}", 'r') as f:
        lines = f.readlines()
    lines = [clean(line) for line in lines if clean(line)!='']
    return lines

In [113]:
def extract_person_dict(lines):
  person = {}
  section="other"
  for i, line in enumerate(lines):
      if "###" in line:
          section = line.replace("- ### ","")
          if section == "Vida" or section == "Documentos":
            person[section] = []
          else:
            person[section] = {}
          pass
      elif section == "Vida":
        person[section].append(clean(line).replace("- ", ""))
      elif section == "Documentos":
        person[section].append(line)
      elif ":" in line:
        vals = line.split(":")
        # print(len(vals))
        if ("Morte" == vals[0] and (clean(vals[1]) == 'DATA, LOCAL')) or ("Sepultamento" == vals[0] and (clean(vals[1]) == 'Cemitério de')):
          person[section][vals[0]] = ''
        elif "Filhos" in vals[0]:
          filhos = []
          for filho in lines[i+1:]:
            if filho[0]=="-":
              filhos.append(filho.replace("- ", ""))
            else:
              break
          person[section]["Filhos"] = filhos
        else:
          person[section][vals[0]] = clean(":".join(vals[1:]))

  person = clean_dict(person)
  # display(person)
  return person

In [114]:
from datetime import date
def create_header(name, dates):
  timest = dates[0] if len(dates)>0 else "????"
  timeend = dates[1] if len(dates)>1 else "????"
  today = date.today().strftime("%d/%m/%Y")
  s = f"---\ntitle: \'{name} ({timest}-{timeend})\'\ndata: {today}\nnome: {name}\nlayout: pessoa\n---\n\n"
  return s

def get_personal_data(personal_data):
  lines = []
  for k, v in personal_data.items():
    if k =="Pais":
      text = v.split(" e ")
      line = f'**{k}:** [{text[0]}](){{:class="underconstruction"}} e [{text[1]}](){{:class="underconstruction"}}<br/>'
      lines.append(line)
    elif k =="Casamento":
      text = v.split(",")
      line = f'**{k}:** [{text[0]}](){{:class="underconstruction"}}, {",".join(text[1:])}<br/>'
      lines.append(line)
    elif k =="Filhos":
      line = f"**{k}:**<br/>"
      lines.append(line)
      # print(v)
      for i, filho in enumerate(v):
        line = f'&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{i+1}. [{filho}](){{:class="underconstruction"}}<br/>'
        lines.append(line)
    else:
      line = f"**{k}:** {v}<br/>"
      lines.append(line)
  return "\n".join(lines)

def get_life(vida):
  nl = '\n' 
  paragraphs = "\n\n".join(vida)
  text = f"{nl}{nl}## Vida:{nl}{nl}{paragraphs}{nl}"
  return text

def get_refs(refs):
  print(refs)
  nl = '\n' 
  paragraphs = "\n\n".join(refs)

  text = f"{nl}{nl}## Referências:{nl}{nl}{paragraphs}{nl}"
  return text

In [115]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir("convert_files") if (isfile(join("convert_files", f)) and f !='.DS_Store')]
onlyfiles

['João Reynaldo Scherer (1876-1946).md',
 'Henrique Scherer Sobrinho (1848-----).md',
 'Waldemar Ziegler Scherer (1914-1989).md',
 'Maria Catharina Scherer (1826-----).md']

In [116]:
def get_time_name(filename):
  filename = filename.replace(".md", '')
  times = re.findall(r'\d+', filename)
  name = " ".join(re.sub(r'\([^)]*\)', '', filename).split())
  return times, name


In [117]:
def file_name_set(name, time):
  name_aux = "".join(name.split())
  date_aux = time[0] if len(time)>0 else ""
  return f"{name_aux}{date_aux}.md"

def save_file(name, time, person):
  f = open(f"converted/{file_name_set(name, time)}", "w")
  f.write(create_header(name, times))
  f.write(get_personal_data(person["Dados Pessoais"]))
  f.write(get_life(person["Vida"]))
  f.write(get_refs(person["Documentos"]))
  f.close()

In [118]:
for filename in onlyfiles:
  times, name = get_time_name(filename)
  lines = read_lines(filename)
  person = extract_person_dict(lines)
  save_file(name, times, person)


['Certidão de Nascimento: ', 'Registro de Batismo: ', 'Certidão de Casamento:', 'Certidão de Óbito: ', '[1] Tumulo', 'Outros:', '- [2] Aparece como marido de Adelaide na Certidão de Óbito de Adelaide Ziegler Scherer (1890-1929) - https://drive.google.com/file/d/1QpM7f7GsNPCXR-5OuPlmgFD33ftCBwlw/view?usp=sharing', '- https://drive.google.com/file/d/1QpM7f7GsNPCXR-5OuPlmgFD33ftCBwlw/view?usp=sharing', '- [3] Aparece como pai de Docelina na Certidão de Nascimento de Docelina Scherer (1908 - ????) - https://www.familysearch.org/ark:/61903/3:1:3QS7-L9L8-6HYQ?cc=3741255&personaUrl=%2Fark%3A%2F61903%2F1%3A1%3ADHS5-HGN2', '- [4] Aparece como pai de Fernando na Certidão de Nascimento de Fernando Scherer (1907-????) -https://www.familysearch.org/ark:/61903/3:1:3QS7-L9L8-6HGF?cc=3741255&personaUrl=%2Fark%3A%2F61903%2F1%3A1%3ADSTY-V9ZM', '- [5] Family Search - Person - No oficial source', '- https://www.familysearch.org/tree/person/details/LB6R-FR6', 'Agradecimento: João Miguel Scherer (JM)']
['Ce

IndexError: list index out of range